## INSTALL DEPENDENCIES

In [ ]:
%pip install faker
%pip install numpy pandas matplotlib seaborn pyarrow
%pip install ipython-sql psycopg2-binary sqlalchemy

%load_ext sql


## Config Database Connection

In [ ]:

username = 'postgres'
password = 'logos_postgres'
hostname = 'localhost'
port = '5432'
database = 'postgres'

connection_string = f'postgresql://{username}:{password}@{hostname}:{port}/{database}'

%sql $connection_string

In [ ]:
# config variables for the database connection
table_name = 'test'

## Doing SQL Commands on Database
- Create table
- Insert mock data
- Get data
- Delete all data from table


In [ ]:
# Create test table

%sql CREATE TABLE IF NOT EXISTS $table_name (\
    id SERIAL PRIMARY KEY,\
    answer TEXT,\
    question TEXT,\
    keywords TEXT[],\
    updatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP\
)

In [ ]:
# Generate mock data
from faker import Faker
import random

fake = Faker()
TOPIC = 'astronomy' # Assume we work on topic of astronomy
ANS_COUNT = 100 # Assume we have 100 distinct answers
QUES_COUNT = 100 # Assume we have 100 distinct questions

keywords = ['star', 'planet', 'galaxy', 'blackhole', 'wormhole', 'nebula', 'supernova', 'quasar', 'pulsar', 'cosmology',
            'lightcone', 'bigbang', 'darkmatter', 'darkenergy', 'gravity', 'relativity', 'quantum', 'relativity', 'telescope']

# Create a dictionary where each question maps to a unique answer and keywords
ques_ans_dict = {fake.text(): (fake.text(), random.sample(keywords, k=random.randint(0, len(keywords)))) for _ in range(QUES_COUNT)}

mock_data = []
DATA_COUNT = 1000 # Number of mock data to be inserted

for i in range(DATA_COUNT):
    ques, (ans, keys) = random.choice(list(ques_ans_dict.items())) # Select a random question-answer-keywords tuple
    mock_data.append((ans, ques, keys))

In [ ]:
# Insert 1 mock data
%sql INSERT INTO $table_name (answer, question, keywords) \
    VALUES ('Saturn', 'What is the 6th planet in the Solar System', array['Saturn', 'planet', 'astronomy'])
;

In [ ]:
# Insert multiple mock data
for data in mock_data:
    answer, question, keywords = data
    keywords_str = ', '.join(f"'{keyword}'" for keyword in keywords) # convert list of keywords to string   
    %sql INSERT INTO $table_name (answer, question, keywords) VALUES (:answer, :question, :keywords)

In [ ]:
# get all data from the table
ans = %sql SELECT * FROM $table_name
print(ans)

In [ ]:
# Delete all data from the table
# %sql TRUNCATE $table_name

# Drop the table
#%sql DROP TABLE $table_name

## Doing Data Analysis on Database

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to next line
pd.set_option('max_colwidth', None)  # Display full content of each field


In [ ]:
# Convert the data to a pandas dataframe
df = pd.DataFrame(ans, columns=ans.keys)

print('Top 5 rows of the dataframe:')
print(df.head())
# print info about the dataframe
print('\nInfo about the dataframe:')
print(df.info())


In [ ]:
# Algorithm to analyze by keywords, meaning for each keyword, we want to know how many questions contain that keyword
# Then visualize a distribution of the number of questions for each keyword

# Input:
# A pandas dataframe df with columns ['id', 'answer', 'question', 'keywords', 'updatedat'], where keywords is a list of keywords

# Output:
# A plot showing the distribution of the number of questions for each keyword

# 1. Loop through rows, take distinct keywords, add to a dict
# Map keyword => ids list
# 2. Loop through rows to find keywords appearences (maybe input it to a dict)
# 3. Plot it

from typing import Dict, List
def construct_kwdict(pandas_df: pd.DataFrame) -> Dict[str, List[int]]:
    kwdict = {}
    for _, row in pandas_df.iterrows(): # each row in df table
        for kw in row["keywords"]: # keywords col of the row
            if kw not in kwdict:
                kwdict[kw] = [row["id"]]
            else:
                kwdict[kw].append(row["id"])
            
    return kwdict

kw_dict = construct_kwdict(df)

print(kw_dict)
print(len(kw_dict))


In [ ]:
# Visualize the distribution of the number of questions for each keyword
import matplotlib.pyplot as plt

kw_list = []
for key in kw_dict.keys():
    kw_list.append(key)

id_count = []
for id_list in kw_dict.values():
    id_count.append(len(id_list))

print(kw_list)
print(id_count)
plt.barh(id_count. kw_list, color = "maroon", width = 0.2)
plt.xlabel('Keywords')
plt.ylabel('Number of questions')
plt.title('Keyword distribution')
plt.savefig('keyword_distribution.png')
plt.show()